In [1]:
import joblib
from tensorflow.keras.models import load_model
import tensorflow as tf
import pandas as pd
from sklearn.metrics import classification_report

2023-06-13 15:38:22.419080: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
# load fitted pipeline
pipeline_loaded = joblib.load('models/fitted_ANN_preprocessing_pipeline.pkl')

claner_loaded = joblib.load('models/cleaner.pkl')

In [3]:
## define f1 score 
#def f1_score_ann(y_true, y_pred):
#    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
#    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
#    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
#
#    precision = true_positives / (predicted_positives + K.epsilon())
#    recall = true_positives / (possible_positives + K.epsilon())
#
#    f1 = 2 * (precision * recall) / (precision + recall + K.epsilon())
#    return f1
#
## load model
## Register the custom metric function within a custom object scope
#custom_objects = {'f1_score_ann': f1_score_ann}
#with tf.keras.utils.custom_object_scope(custom_objects):
#    # Load the saved model
#    loaded_model = load_model('models/ann.h5')
#
#

In [4]:
loaded_model = tf.keras.models.load_model('models/ann.h5', compile=False)

2023-06-13 15:38:23.803839: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [5]:
#load data

X_test = pd.read_csv('data/X_test.csv')
#X_test.head()
print(X_test.isna().sum())

y_test = pd.read_csv('data/y_test.csv')

Profile_mean          0
Profile_std           0
Profile_kurtosis    252
Profile_skewness      0
DM_SNR_mean           0
DM_SNR_std          179
DM_SNR_kurtosis       0
DM_SNR_skewness     103
dtype: int64


In [6]:
# apply cleaning
X_test_clean = pd.DataFrame(claner_loaded.transform(X_test), columns=X_test.columns)
X_test_clean.head()

,Profile_mean,Profile_std,Profile_kurtosis,Profile_skewness,DM_SNR_mean,DM_SNR_std,DM_SNR_kurtosis,DM_SNR_skewness
0,106.015625,50.244711,0.308204,0.274101,6.869565,32.528835,5.124209,26.443823
1,109.562500,36.626227,0.278254,1.814582,5.351171,25.645325,5.864311,38.502413
2,119.445312,46.846126,0.179741,0.111426,3.415552,19.284797,6.680946,52.471406
3,108.468750,44.300278,0.234723,0.737700,1.494983,12.326351,12.454074,196.299397
4,26.578125,31.823637,5.502087,31.529843,134.798495,72.055583,0.029297,-1.211265


In [7]:
# apply preprocessing
X_test_preprocessed = pipeline_loaded.transform(X_test_clean)

In [8]:
# Use the loaded model for predictions
predictions = (loaded_model.predict(X_test_preprocessed) > 0.5).astype('int8')
print(classification_report(y_test, predictions))


#y_pred = (model_ann.predict(X_test_preprocessed) > 0.5).astype('int8')
#y_pred_proba = model_ann.predict(X_test_preprocessed)
#
#y_true = y_test
#
##evaluate
#print('Evaluation-----------------')
#print('Recall :', recall_score(y_true, y_pred))
#print('Precision :', precision_score(y_true, y_pred))
#print('F1 :', f1_score(y_true, y_pred))
#print('Accuracy :', accuracy_score(y_true, y_pred))
#print('ROC_AUC :', roc_auc_score(y_true, y_pred_proba))
#

59/59 [==============================] - 0s 372us/step
              precision    recall  f1-score   support

         0.0       0.99      1.00      0.99      1708
         1.0       0.96      0.87      0.91       171

    accuracy                           0.98      1879
   macro avg       0.97      0.93      0.95      1879
weighted avg       0.98      0.98      0.98      1879

